In [1]:
# Initialize librarys
from scipy.special import erfc, erf
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import numpy as np
import math
import json
from ipywidgets import *
import sys


In [2]:
label_calc = "Berechnetes Druckpotential"
label_syn = "Beobachtetes Druckpotential"
ylabel = "Druckpotential" + " (m)"
title_ax1 = "Potentialhöhe entlang x"
title_ax2 = "Lineare Regression"

def calculate_head(hr, hl, l1, l2, k1, k2, res):
    """
    Calculate the head for 1D unconfined flow with two Dirichlet B.C. and 
    two sections of different hydraulic conductivity
    
    Keyword Arguments:
    hr -- head at x=0, m
    hl -- head at x=l1+l2, m
    l1 -- length of k1 section, m
    l2 -- length of k2 section, m
    k1 -- hydraulic conductivity 1, m/s
    k2 -- hydraulic conductivity 2, m/s
    res -- minimum resulution for x array
    """
    q = (hl**2 - hr**2) / (2 * (l1/k1 + l2/k2))
    h_between = np.sqrt(-2*q*l1/k1 + hl**2)
    x1 = np.arange(0,l1, res)
    h_1 = np.sqrt(hl**2 - x1 * (hl**2 - h_between**2) / l1)
    x2 = np.arange(0, l2, res)
    h_2 = np.sqrt(h_between**2 - x2 * (h_between**2 - hr**2) / l2)
    h_tot = np.concatenate((h_1, h_2))
    x_tot = np.concatenate((x1, x2 + l1))
    return x_tot, h_tot


def head(hl, hr, l1, k1, k2, add_syn_data):
    """
    Plot the head and correlation
    
    Keyword Arguments:
    R -- Recharge, mm/a
    hr -- head at x=0, m
    hl -- head at x=L, m
    K -- hydraulic conductivity, m/s
    L -- distance to hl, m
    y_scale -- scale of y, -
    add_syn_data -- add synthetic data to plot, -
    """
    L = 200
    l2 = L - l1
    x, h = calculate_head(hr, hl, l1, l2, k1, k2, 0.1)
    
    # Prepare figure and subplots
    if add_syn_data:
        fig = plt.figure(figsize=(9,18))
        ax = fig.add_subplot(3, 1, 1)
    else:
        fig = plt.figure(figsize=(9,6))
        ax = fig.add_subplot(1, 1, 1)
        
        
    ax.add_patch(Rectangle((0, 0), l1, 100,
             facecolor = 'orange',
             fill=True,
             lw=0))
    ax.add_patch(Rectangle((l1, 0), l2, 100,
         facecolor = 'goldenrod',
         fill=True,
         lw=0))
    ax.annotate("K1", (l1/2, 50), color='black', weight='bold', fontsize=15, ha='center', va='center')
    ax.annotate("K2", (l1 + l2/2, 50), color='black', weight='bold', fontsize=15, ha='center', va='center')
    
    # Plot the head
    ax.plot(x,h, label=label_calc)
    ax.set(xlabel='x (m)', ylabel=ylabel,title=title_ax1)
    
    # Plot the CH-boundaries
    ax.vlines(0, 0, hl, linewidth = 2, color='b')
    ax.vlines(l1+l2, 0, hr, linewidth = 2, color='b')
    
    idx_l1 = int(np.where(x == l1)[0]) + 1
    
    ax.set_xlim(0, l1+l2)
    ax.set_ylim(0 , 100)
    #ax.fill_between(x[:idx_l1],0,h[:idx_l1], facecolor='')
    #ax.fill_between(x[idx_l1:],0,h[idx_l1:], facecolor='')

    # Set y min und max
    #y_min = hl*(1-y_scale/100)
    #y_max = hr*(1+y_scale/100)
    
    
    # Add synthetic data
    if add_syn_data:
        # Calculate synthetic head with mock data
        
        x_syn, h_syn = calculate_head(hr=50, hl=90, l1=50, l2=150, k1=2e-4, k2=3e-5, res=5)
        
        # Plot synthetic head
        ax.scatter(x_syn, h_syn, label=label_syn)

        # Calculate regression parameter
        h_inter = np.interp(x_syn, x, h)
        
        # Add regression to figure
        ax2 = fig.add_subplot(3, 1, 2)
        ax2.scatter(h_syn, h_inter)
        m, b = np.polyfit(h_syn, h_inter, 1)
        #if np.all(np.sort(h_syn) == h_syn):
        #    x_to_plot = np.linspace(min(h),2, 100)
        #    ax2.plot(h_syn, m*x_syn+b)
        rmse = np.sqrt(((h_syn - h_inter) ** 2).mean())
        ax2.annotate(f"RMSE: {round(rmse, 3)}", xy=(0.05, 0.95), xycoords='axes fraction', fontsize=15)
        #ax2.text(min(h_syn)*1.005, max(h_inter)*0.999, , horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='azure'),fontsize=12)
        
        ax2.set(xlabel=label_syn + " (m)", ylabel=label_calc + " (m)",title=title_ax2)
        ax2.grid()
        
        rmse_track.append(rmse)
        k1_track.append(k1)
        k2_track.append(k2)
        ax3 = fig.add_subplot(3, 1, 3)
        sc = ax3.scatter(k1_track, k2_track, c=rmse_track, cmap="viridis")
        ax3.set_yscale('log')
        ax3.set_xscale('log')
        ax3.set_xlim(1e-2, 1e-6)
        ax3.set_ylim(1e-2, 1e-6)
        cb = plt.colorbar(sc)
        cb.ax.set_ylabel("RMSE")
        
        
        ax3.set(xlabel="K1", ylabel="K2",title="RMSE der bisherigen K-WErte")
        ax3.grid()
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


In [3]:
rmse_track = []
k1_track = []
k2_track = []

iap = interactive(head,
         add_syn_data=widgets.Checkbox(False, description="Beobachtung hinzufügen"),
         hl=widgets.BoundedFloatText(value=70, min=0, max=100, step=1, description='H_l:', disabled=False),
         hr=widgets.BoundedFloatText(value=30, min=0, max=100, step=1, description='H_r:', disabled=False),
         l1= widgets.BoundedFloatText(value=50,min=0, max=200,step=1, description='L:' , disabled=False),
         k1=widgets.FloatLogSlider(value=0.0001,base=10,min=-6, max=-2, step=0.1,description='K1:',readout=True,readout_format='.2e', 
                                  continuous_update=False),
         k2=widgets.FloatLogSlider(value=0.0001,base=10,min=-6, max=-2, step=0.1,description='K2:',readout=True,readout_format='.2e', 
                                  continuous_update=False))
output = iap.children[-1]
output.layout.height = '1500px'
iap

interactive(children=(BoundedFloatText(value=70.0, description='H_l:', step=1.0), BoundedFloatText(value=30.0,…

<hr>
&copy; 2022 | Thomas Reimann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>